In [0]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
image_dim=32

In [0]:
def parse_function(filenames):
    #print(filename , label)
    image_string  = tf.read_file(filenames)

    #dont use tf.image.decode_image , or the output shape will be undefined
    image         = tf.image.decode_png(image_string , channels = 3)

    #This will convert to float values in [0 , 1]
    image         = tf.image.convert_image_dtype(image , tf.float32)

    resized_image = tf.image.resize_images(image , [image_dim , image_dim])
    return resized_image 

def train_preprocess(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image , max_delta = 32.0/255.0)
    image = tf.image.random_saturation(image , lower = 0.5 , upper = 1.5)

    #make sure the image is still in [0 , 1]
    image = tf.clip_by_value(image , 0.0 , 1.0)
    return image

In [0]:
def read_data(filenames):
    dataset  = tf.data.Dataset.from_tensor_slices((filenames))
    dataset  = dataset.map(parse_function)
    dataset  = dataset.map(train_preprocess)

    iterator = dataset.make_one_shot_iterator()
    return iterator

In [0]:
def build_training(filenames):
    X            = np.zeros((len(filenames) , image_dim , image_dim , 3))
    iterator     = read_data(filenames)
    next_element = iterator.get_next()
    
    with tf.Session() as sess :
        for i in range (len(filenames)):
            image_example = sess.run(next_element)
            X[i , : , :]  = image_example[: , : , :]
    return X

In [0]:
filenames = ['A_1.PNG','A_2.PNG','A_3.PNG','A_4.PNG','A_5.PNG','A_6.PNG','A_7.PNG','A_8.PNG','A_9.PNG','A_10.PNG', 
             'B_1.PNG','B_2.PNG','B_3.PNG','B_4.PNG','B_5.PNG','B_6.PNG','B_7.PNG','B_8.PNG','B_9.PNG','B_10.PNG',
             'C_1.PNG','C_2.PNG','C_3.PNG','C_4.PNG','C_5.PNG','C_6.PNG','C_7.PNG','C_8.PNG','C_9.PNG','C_10.PNG',
             'D_1.PNG','D_2.PNG','D_3.PNG','D_4.PNG','D_5.PNG','D_6.PNG','D_7.PNG','D_8.PNG','D_9.PNG','D_10.PNG',
             'E_1.PNG','E_2.PNG','E_3.PNG','E_4.PNG','E_5.PNG','E_6.PNG','E_7.PNG','E_8.PNG','E_9.PNG','E_10.PNG',
             'F_1.PNG','F_2.PNG','F_3.PNG','F_4.PNG','F_5.PNG','F_6.PNG','F_7.PNG','F_8.PNG','F_9.PNG','F_10.PNG',
             'G_1.PNG','G_2.PNG','G_3.PNG','G_4.PNG','G_5.PNG','G_6.PNG','G_7.PNG','G_8.PNG','G_9.PNG','G_10.PNG',
             'I_1.PNG','I_2.PNG','I_3.PNG','I_4.PNG','I_5.PNG','I_6.PNG','I_7.PNG','I_8.PNG','I_9.PNG','I_10.PNG',
             'K_1.PNG','K_2.PNG','K_3.PNG','K_4.PNG','K_5.PNG','K_6.PNG','K_7.PNG','K_8.PNG','K_9.PNG','K_10.PNG',
             'L_1.PNG','L_2.PNG','L_3.PNG','L_4.PNG','L_5.PNG','L_6.PNG','L_7.PNG','L_8.PNG','L_9.PNG','L_10.PNG']

Y         = np.array([0,0,0,0,0,0,0,0,0,0,
            1,1,1,1,1,1,1,1,1,1,
            2,2,2,2,2,2,2,2,2,2,
            3,3,3,3,3,3,3,3,3,3,
            4,4,4,4,4,4,4,4,4,4,
            5,5,5,5,5,5,5,5,5,5,
            6,6,6,6,6,6,6,6,6,6,
            7,7,7,7,7,7,7,7,7,7,
            8,8,8,8,8,8,8,8,8,8,
            9,9,9,9,9,9,9,9,9,9])

X         = build_training(filenames)

model     = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(image_dim*image_dim*3 , activation = tf.nn.relu),
           
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation = tf.nn.softmax)
            ]) 

model.compile(optimizer = 'adam' ,
              loss      = 'sparse_categorical_crossentropy',
              metrics   = ['accuracy'])
model.fit(X,Y,epochs=2500,batch_size=100)

model.summary()

model.evaluate(X,Y)

Train on 100 samples
Epoch 1/2500
100/100 [==============================] - 0s 3ms/sample - loss: 2.6213 - acc: 0.0800
Epoch 2/2500
100/100 [==============================] - 0s 974us/sample - loss: 12.5850 - acc: 0.1100
Epoch 3/2500
100/100 [==============================] - 0s 931us/sample - loss: 16.2316 - acc: 0.1000
Epoch 4/2500
100/100 [==============================] - 0s 1ms/sample - loss: 19.9773 - acc: 0.1000
Epoch 5/2500
100/100 [==============================] - 0s 1ms/sample - loss: 19.1171 - acc: 0.1000
Epoch 6/2500
100/100 [==============================] - 0s 991us/sample - loss: 18.8574 - acc: 0.0900
Epoch 7/2500
100/100 [==============================] - 0s 889us/sample - loss: 16.3969 - acc: 0.0800
Epoch 8/2500
100/100 [==============================] - 0s 1ms/sample - loss: 13.2226 - acc: 0.1200
Epoch 9/2500
100/100 [==============================] - 0s 1ms/sample - loss: 10.5855 - acc: 0.1000
Epoch 10/2500
100/100 [==============================] - 0s 918us/sample

[0.0035046343412250282, 1.0]